## Imports

In [4]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from snowflake.ml.modeling.preprocessing import *
from snowflake.ml.modeling.impute import *
# from snowflake.snowpark.functions import udf

import sys
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from itertools import chain, combinations

## Snowflake Details / Connection

In [2]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

In [3]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "ADTRACKING"."ADTRACKING_SCHEMA"
Current Warehouse: "AMAZON_SAGEMAKE_W_SNOWFLAKE_AS_DATASOURCE"


In [4]:
snowpark_df_train = session.table("ADTRACKING_CLICKS_TRAIN")
snowpark_df_test = session.table("ADTRACKING_CLICKS_TEST")

## Creating model dataframes
We know what we expect to work from our 01_snowpark_eda notebook.  The same notebook also gave some ideas on possible features that were not tested that can easily be added to our model and further pruned.  

Addendum: Reading through some of the higher rated kaggle notebooks also gave plenty of new ideas to add.  A lot of these will take more work than there is currently time for, but some can be added relatively easily and will be below.  See https://www.kaggle.com/code/nanomathias/feature-engineering-importance-testing for simple, easy-to-add features that we can test with.  We can replicate their work but also expand it to attempt similar LGB equivalents.  Testing both could be interesting.  I would expect the XGBoost approach they used to overfit a bit and take longer to run as compared to an equivalent LGB model.

Since we want to compare their approach with an LGB model, let us first create the requisite features from the above linked notebook.  

Note that here we will just be comparing their model to LGB.  Future notebooks will be added (or perhaps hosted on AWS Sagemaker) with different features

We will also be combining Nanomathias' feature engineering with some of the approaches from https://www.kaggle.com/competitions/talkingdata-adtracking-fraud-detection/discussion/56475
Specifically the down-sampling approach and their approach to binning the training/validation data.  

The goal here is to get a simple model based off of easy to create and understand features that trains quickly.  A sort of 'good enough' approach as a first pass due to the short amount of time on this project. (Est. 2 days max)

In [7]:
snowpark_df_train.columns

['IP',
 'APP',
 'DEVICE',
 'OS',
 'CHANNEL',
 'CLICK_TIME',
 'ATTRIBUTED_TIME',
 'IS_ATTRIBUTED']

In [8]:
snowpark_df_test.columns

['CLICK_ID', 'IP', 'APP', 'DEVICE', 'OS', 'CHANNEL', 'CLICK_TIME']

### Adding features from Nanomathias' notebook
Along with their features, we will use all the combinations of features as shown in Komaki's notebook (second link above).  This will increase training time, but not too much due to our negative down sampling.

In [16]:
# Add time columns
snowpark_df_train = snowpark_df_train.with_column("DAY", F.date_part("day", F.col("CLICK_TIME")))
snowpark_df_train = snowpark_df_train.with_column("HOUR", F.date_part("hour", F.col("CLICK_TIME")))
snowpark_df_train = snowpark_df_train.with_column("MINUTE", F.date_part("minute", F.col("CLICK_TIME")))
snowpark_df_train = snowpark_df_train.with_column("SECOND", F.date_part("second", F.col("CLICK_TIME")))

Note our training data is large, but not so large that it can't be kept fully in memory.  If it was, we could simply batch in X rows e.g. 100k/batch or whatever was needed to keep in mem.
Then we would put the following _CONFRATE calculations into a function.  For now we'll keep it as is.  If we end up coming back to this after the 2 days are up, we can revisit optimizing this for larger datasets.

In any case, since we are planning on doing negative downsampling we can simply reduce the size before read-in with that.

Komaki's notebook shows that negative down sampling such that the sizes are equal are more than up for the challenge.  The hope is that it can still perform well even with many, many fewer features as compared to their notebook.  If time allows in the future, we may try a SMOTE approach as well and compare.

In [178]:
# Perform our negative downsampling

# First select all our rows where is_attributed == 1
positive_trains = snowpark_df_train.filter(F.col("IS_ATTRIBUTED") == 1)
# print(positive_trains.count()) # 456846

# Get our negative cases separated
negative_trains = snowpark_df_train.filter(F.col("IS_ATTRIBUTED") == 0)
# Randomly sample our negative cases 
small_negative_trains = negative_trains.sample(n=positive_trains.count())
print(f"There are {positive_trains.count()} positive samples and we downsampled to {small_negative_trains.count()} negative samples")

There are 456846 positive samples and we downsampled to 456846 negative samples


In [191]:
# Join the downsampled negatives with our full suite of positives
downsampled_trains = positive_trains.natural_join(small_negative_trains, join_type="full")
print(downsampled_trains.count())

913692


In [193]:
# Convert the downsampled training data in to a pandas df for further manipulation
train_df = downsampled_trains.to_pandas()

In [194]:
train_df.head()

,IP,APP,DEVICE,OS,CHANNEL,CLICK_TIME,ATTRIBUTED_TIME,IS_ATTRIBUTED,DAY,HOUR,MINUTE,SECOND
0,151462,26,1,19,266,2017-11-07 07:33:18,NaT,0,7,7,33,18
1,249327,14,1,9,349,2017-11-09 09:08:21,NaT,0,9,9,8,21
2,123945,8,1,8,145,2017-11-07 23:39:14,NaT,0,7,23,39,14
3,88552,12,1,19,265,2017-11-07 07:10:49,NaT,0,7,7,10,49
4,63340,15,1,19,386,2017-11-09 03:37:51,NaT,0,9,3,37,51


In [2]:
# Get all combos of our base categories
base_cats = ["IP", "APP", "DEVICE", "OS", "CHANNEL"]
def powerset(iter):
    s = list(iter)
    return [list(comb) for r in range(len(s)+1) for comb in combinations(s, r) if len(comb) > 0] # Don't keep the empty set

In [195]:
def rate_calc(x, log_group=np.log(100000)):
    rate = x.sum() / float(x.count())
    conf = np.min([1, np.log(x.count()) / log_group])
    return rate * conf

In [196]:
for cols in powerset(base_cats):
    new_feature = "_".join(cols)+"_CONFRATE"
    group_obj = train_df.groupby(cols)
    
    group_sizes = group_obj.size()
    print(f"Calculating confidence rate for: {cols} \n"
          f"Saving to: {new_feature}.  Group Max / Mean / Median / Min: {group_sizes.max()} / {np.round(group_sizes.mean(), 2)} / {np.round(group_sizes.median(), 2)} / {group_sizes.min()}")
    
    train_df = train_df.merge(
        group_obj['IS_ATTRIBUTED'].
            apply(rate_calc).
            reset_index().
            rename(index=str, 
                   columns={"IS_ATTRIBUTED":new_feature}
            )[cols + [new_feature]],
        on=cols, how='left'
    )

Calculating confidence rate for: ['IP'] 
Saving to: IP_CONFRATE.  Group Max / Mean / Median / Min: 5416 / 3.61 / 1.0 / 1
Calculating confidence rate for: ['APP'] 
Saving to: APP_CONFRATE.  Group Max / Mean / Median / Min: 134341 / 2760.4 / 16.0 / 1
Calculating confidence rate for: ['DEVICE'] 
Saving to: DEVICE_CONFRATE.  Group Max / Mean / Median / Min: 737505 / 485.23 / 2.0 / 1
Calculating confidence rate for: ['OS'] 
Saving to: OS_CONFRATE.  Group Max / Mean / Median / Min: 188836 / 4860.06 / 20.0 / 1
Calculating confidence rate for: ['CHANNEL'] 
Saving to: CHANNEL_CONFRATE.  Group Max / Mean / Median / Min: 134640 / 5076.07 / 1527.0 / 1
Calculating confidence rate for: ['IP', 'APP'] 
Saving to: IP_APP_CONFRATE.  Group Max / Mean / Median / Min: 921 / 1.64 / 1.0 / 1
Calculating confidence rate for: ['IP', 'DEVICE'] 
Saving to: IP_DEVICE_CONFRATE.  Group Max / Mean / Median / Min: 3927 / 3.0 / 1.0 / 1
Calculating confidence rate for: ['IP', 'OS'] 
Saving to: IP_OS_CONFRATE.  Group Max

In [197]:
train_df.head()

,IP,APP,DEVICE,OS,CHANNEL,CLICK_TIME,ATTRIBUTED_TIME,IS_ATTRIBUTED,DAY,HOUR,...,APP_DEVICE_OS_CONFRATE,APP_DEVICE_CHANNEL_CONFRATE,APP_OS_CHANNEL_CONFRATE,DEVICE_OS_CHANNEL_CONFRATE,IP_APP_DEVICE_OS_CONFRATE,IP_APP_DEVICE_CHANNEL_CONFRATE,IP_APP_OS_CHANNEL_CONFRATE,IP_DEVICE_OS_CHANNEL_CONFRATE,APP_DEVICE_OS_CHANNEL_CONFRATE,IP_APP_DEVICE_OS_CHANNEL_CONFRATE
0,151462,26,1,19,266,2017-11-07 07:33:18,NaT,0,7,7,...,0.135747,0.156350,0.123234,0.123774,0.0,0.000000,0.0,0.0,0.123774,0.0
1,249327,14,1,9,349,2017-11-09 09:08:21,NaT,0,9,9,...,0.007903,0.061975,0.009009,0.007730,0.0,0.000000,0.0,0.0,0.009009,0.0
2,123945,8,1,8,145,2017-11-07 23:39:14,NaT,0,7,23,...,0.132937,0.342800,0.125584,0.127659,0.0,0.030103,0.0,0.0,0.129202,0.0
3,88552,12,1,19,265,2017-11-07 07:10:49,NaT,0,7,7,...,0.052196,0.035021,0.046451,0.050982,0.0,0.000000,0.0,0.0,0.048011,0.0
4,63340,15,1,19,386,2017-11-09 03:37:51,NaT,0,9,3,...,0.062452,0.055411,0.061086,0.061336,0.0,0.000000,0.0,0.0,0.061336,0.0


In [199]:
# Store this iteration locally for testing
# train_df.to_csv("data/after_rate_calc.csv")

In [31]:
# train_df = pd.read_csv("data/after_rate_calc.csv", index_col=False, parse_dates=['CLICK_TIME', 'ATTRIBUTED_TIME']).drop(columns=["Unnamed: 0"])

In [32]:
# train_df.head()

In [33]:
# train_df.dtypes

We'll add in the 'NEXTCLICK' feature which shows great predictive power across multiple notebooks

In [34]:
D = 2**26
# Get the closest we can to a unique user given the data and apply a hash
train_df['CATEGORY'] = (train_df['IP'].astype(str) + '_' + train_df['APP'].astype(str) + '_' + train_df['DEVICE'].astype(str) \
    + '_' + train_df['OS'].astype(str)).apply(hash) % D

click_buffer = np.full(D, 3000000000, dtype=np.uint32)

train_df['EPOCHTIME'] = train_df['CLICK_TIME'].astype(np.int64) // 10 ** 9

next_clicks = []
for category, t in zip(reversed(train_df['CATEGORY'].values), reversed(train_df['EPOCHTIME'].values)):
    next_clicks.append(click_buffer[category]-t)
    click_buffer[category] = t
del(click_buffer)

nextclick_times = list(reversed(next_clicks))

train_df['NEXTCLICK'] = nextclick_times

In [35]:
train_df.head()

,IP,APP,DEVICE,OS,CHANNEL,CLICK_TIME,ATTRIBUTED_TIME,IS_ATTRIBUTED,DAY,HOUR,...,DEVICE_OS_CHANNEL_CONFRATE,IP_APP_DEVICE_OS_CONFRATE,IP_APP_DEVICE_CHANNEL_CONFRATE,IP_APP_OS_CHANNEL_CONFRATE,IP_DEVICE_OS_CHANNEL_CONFRATE,APP_DEVICE_OS_CHANNEL_CONFRATE,IP_APP_DEVICE_OS_CHANNEL_CONFRATE,CATEGORY,EPOCHTIME,NEXTCLICK
0,151462,26,1,19,266,2017-11-07 07:33:18,NaT,0,7,7,...,0.123774,0.0,0.000000,0.0,0.0,0.123774,0.0,43158837,1510039998,1489960002
1,249327,14,1,9,349,2017-11-09 09:08:21,NaT,0,9,9,...,0.007730,0.0,0.000000,0.0,0.0,0.009009,0.0,67037707,1510218501,1489781499
2,123945,8,1,8,145,2017-11-07 23:39:14,NaT,0,7,23,...,0.127659,0.0,0.030103,0.0,0.0,0.129202,0.0,56167569,1510097954,1489902046
3,88552,12,1,19,265,2017-11-07 07:10:49,NaT,0,7,7,...,0.050982,0.0,0.000000,0.0,0.0,0.048011,0.0,40716705,1510038649,1489961351
4,63340,15,1,19,386,2017-11-09 03:37:51,NaT,0,9,3,...,0.061336,0.0,0.000000,0.0,0.0,0.061336,0.0,18677241,1510198671,1489801329


Now we'll add some of the groupby aggregations present in Nanomathias' notebook as well as well as adding in a lot of extra groupby comparisons to test our model with.

We remove their V2 groupby feature and replace it with a more distinct groupby.  Just using one identifier does not guarantee a unique user at all, especially since IP is not a true IP but rather grouped in the raw data.

The https://www.kaggle.com/code/bk0000/non-blending-lightgbm-model-lb-0-977/script notebook has some feature importances listed out (pasted below)

From this we can see that in their model 'day'' is unimportant and probably could be removed if it wasn't used in the creation of another, good predictor. We see that the features 'nextClick', 'hour', 'ip_tcount', 'ip_app_count', 'ip_app_os_count', and 'ip_app_os_var' are all created features that add some value to their model.

In [15]:
# names = ['nextClick', 'nextClick_shift', 'app', 'device', 'os', 'channel', 'hour', 'day', 'ip_tcount', 'ip_tchan_count', 'ip_app_count', 'ip_app_os_count', 'ip_app_os_var', 'ip_app_channel_var_day', 'ip_app_channel_mean_hour', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']
# importances = [138, 1, 232, 27, 147, 305, 105, 0, 40, 2, 32, 21, 10, 7, 8, 52, 9, 9, 33, 11, 28, 13, 7, 35]
# sorted(list(zip(names, importances)), key=lambda x: x[1])

In [7]:
for cols in powerset(base_cats):
    if len(cols) == 5:
        print(cols)

['IP', 'APP', 'DEVICE', 'OS', 'CHANNEL']


In [ ]:
groupby_aggs = []
# These are some of the most influentual features from Baris Kanber's notebook
groupby_aggs.append({'groupby': ['IP', 'DAY', 'HOUR'], 'select': 'CHANNEL', 'agg': 'count'})
groupby_aggs.append({'groupby': ['IP', 'APP'], 'select': 'CHANNEL', 'agg': 'count'})
groupby_aggs.append({'groupby': ['IP', 'APP', 'OS'], 'select': 'CHANNEL', 'agg': 'count'})
groupby_aggs.append({'groupby': ['IP', 'APP', 'OS'], 'select': 'HOUR', 'agg': 'var'})
# These are some influental features from Nanomathias' notebook
groupby_aggs.append({'groupby': ['IP'], 'select': 'APP', 'agg': 'nunique'})


#IP_DAY_HOUR_COUNT_CHANNEL, IP_OS_DEVICE_APP_NEXTCLICK, IP_APP_NEXTCLICK, (REQ. NEXTCLICK anyways, can see how from Kanber's nb)

In [13]:

# for cols in powerset(base_cats):
#     if cols == ['IP']:
#         groupby_aggs.append({'groupby': cols, 'select': 'CHANNEL', 'agg': 'nunique'})
#         groupby_aggs.append({'groupby': cols, 'select': 'APP', 'agg': 'nunique'})
#         groupby_aggs.append({'groupby': cols, 'select': 'DEVICE', 'agg': 'nunique'})
#         groupby_aggs.append({'groupby': cols, 'select': 'APP', 'agg': 'cumcount'})
#         groupby_aggs.append({'groupby': cols, 'select': 'OS', 'agg': 'cumcount'})
        
#     if cols == ['APP']:
#         groupby_aggs.append({'groupby': cols, 'select': 'CHANNEL', 'agg': 'nunique'})
#         groupby_aggs.append({'groupby': cols, 'select': 'DEVICE', 'agg': 'nunique'})

#     if cols == ['IP', 'APP']:
#         groupby_aggs.append({'groupby': cols, 'select': 'CHANNEL', 'agg': 'count'})
        
#     if cols == ['IP', 'APP', 'CHANNEL']:
#         groupby_aggs.append({'groupby': cols, 'select': 'DAY', 'agg': 'var'})
#         groupby_aggs.append({'groupby': cols, 'select': 'HOUR', 'agg': 'var'})
#         groupby_aggs.append({'groupby': cols, 'select': 'HOUR', 'agg': 'mean'})

In [ ]:
groupby_aggs = [
    
    # Variance in day, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'day', 'agg': 'var'},
    ## Variance in hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'var'},
    ## Variance in minute, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'minute', 'agg': 'var'},
    # Variance in hour, for ip-app-os
    {'groupby': ['ip','app','os'], 'select': 'hour', 'agg': 'var'},
    # Variance in hour, for ip-day-channel
    {'groupby': ['ip','day','channel'], 'select': 'hour', 'agg': 'var'},
    # Count, for ip-day-hour
    {'groupby': ['ip','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'mean'}, 
    
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: float(len(x)) / len(x.unique()), 
     'agg_name': 'AvgViewPerDistinct'
    },
    # How popular is the app or channel?
    {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'app', 'agg': 'count'},
    
    {'groupby': ['ip'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'app', 'agg': 'nunique'}, 
    {'groupby': ['ip','day'], 'select': 'hour', 'agg': 'nunique'}, 
    {'groupby': ['ip','app'], 'select': 'os', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'device', 'agg': 'nunique'}, 
    {'groupby': ['app'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip', 'device', 'os'], 'select': 'app', 'agg': 'nunique'}, 
    {'groupby': ['ip','device','os'], 'select': 'app', 'agg': 'cumcount'}, 
    {'groupby': ['ip'], 'select': 'app', 'agg': 'cumcount'}, 
    {'groupby': ['ip'], 'select': 'os', 'agg': 'cumcount'}, 
    {'groupby': ['ip','day','channel'], 'select': 'hour', 'agg': 'var'}    
]